In [3]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# fix 'DirectoryIterator' object has no attribute 'shape'
from tensorflow.python.keras.utils.data_utils import Sequence

import warnings
warnings.filterwarnings('ignore')

#### Cargar Imagenes del Disco Duro con ImageDataGenerator

In [2]:
ENTRENAMIENTO_DIR = "data/CK/"
PRUEBAS_DIR = "data/CK_PRUEBAS/"

# configuracion de entrenamiento
entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# generador
generador_entrenamiento = entrenamiento_datagen.flow_from_directory(
    ENTRENAMIENTO_DIR,
    target_size=(48,48),
    class_mode='categorical'
)

# configuacion de pruebas (test set)
pruebas_datagen = ImageDataGenerator(rescale=1./255)

# generador
generator_pruebas = pruebas_datagen.flow_from_directory(
        PRUEBAS_DIR,
        target_size=(48, 48),
        class_mode='categorical')

Found 911 images belonging to 7 classes.
Found 70 images belonging to 7 classes.


#### layers.BatchNormalization()

La normalización por lotes aplica una transformación que mantiene la salida media cercana a 0 y la desviación estándar de salida cercana a 1.

In [4]:
modelo = keras.Sequential(
    [
        keras.Input(shape=(48, 48, 3)),
        # =============================
        # ...CAPAS CONVOLUCIONALES...
        # =============================
        layers.Flatten(),
        layers.BatchNormalization(),
        layers.Dense(2048, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(1024, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(200, activation="relu"),
        layers.Dense(7, activation="softmax"),
    ]
)

modelo.build((48, 48))
modelo.summary()
modelo.compile(loss = 'categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 6912)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 6912)              27648     
_________________________________________________________________
dense (Dense)                (None, 2048)              14157824  
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
____

In [ ]:
historia = modelo.fit(
    generador_entrenamiento,
    steps_per_epoch = 100,
    epochs = 300, 
    verbose = 1,
    validation_data=generator_pruebas,
    validation_steps=100)